In [33]:
import requests
import os
from datetime import datetime

MASSIVE_API_KEY = os.getenv('MASSIVE_API_KEY')

# Your exact request
start_ms = int(datetime(2025, 11, 20, 14, 30, 0).timestamp() * 1000)
end_ms = int(datetime(2025, 11, 20, 21, 0, 0).timestamp() * 1000)

url = f"https://api.massive.com/v2/aggs/ticker/NVDA/range/5/minute/{start_ms}/{end_ms}"

print(f"URL: {url}")
print(f"Start: {start_ms} = {datetime.fromtimestamp(start_ms/1000)}")
print(f"End: {end_ms} = {datetime.fromtimestamp(end_ms/1000)}")

headers = {"Authorization": f"Bearer {MASSIVE_API_KEY}"}
params = {"adjusted": "true", "sort": "asc", "limit": 50000}

response = requests.get(url, headers=headers, params=params, timeout=30)
data = response.json()

print(f"\nAPI Response:")
print(f"Status: {data.get('status')}")
print(f"Results count: {data.get('resultsCount')}")
print(f"Query count: {data.get('queryCount')}")

if 'results' in data and len(data['results']) > 0:
    first = data['results'][0]
    last = data['results'][-1]
    
    print(f"\nFirst result:")
    print(f"  Timestamp (ms): {first['t']}")
    print(f"  DateTime: {datetime.fromtimestamp(first['t']/1000)}")
    
    print(f"\nLast result:")
    print(f"  Timestamp (ms): {last['t']}")
    print(f"  DateTime: {datetime.fromtimestamp(last['t']/1000)}")
    
    print(f"\nMissing time: {(datetime.fromtimestamp(first['t']/1000) - datetime(2025, 11, 20, 14, 30, 0)).total_seconds() / 3600:.2f} hours")

URL: https://api.massive.com/v2/aggs/ticker/NVDA/range/5/minute/1763620200000/1763643600000
Start: 1763620200000 = 2025-11-20 14:30:00
End: 1763643600000 = 2025-11-20 21:00:00

API Response:
Status: OK
Results count: 49
Query count: 245

First result:
  Timestamp (ms): 1763629200000
  DateTime: 2025-11-20 17:00:00

Last result:
  Timestamp (ms): 1763643600000
  DateTime: 2025-11-20 21:00:00

Missing time: 2.50 hours


In [35]:
start_ms = int(datetime(2025, 11, 13, 14, 30, 0).timestamp() * 1000)
end_ms = int(datetime(2025, 11, 13, 21, 0, 0).timestamp() * 1000)

url = f"https://api.massive.com/v2/aggs/ticker/NVDA/range/5/minute/{start_ms}/{end_ms}"

response = requests.get(url, headers=headers, params=params, timeout=30)
data = response.json()

print(f"Nov 13 Results: {data.get('resultsCount')}")
if data.get('results'):
    first_ts = data['results'][0]['t']
    print(f"First bar: {datetime.fromtimestamp(first_ts/1000)}")

Nov 13 Results: 49
First bar: 2025-11-13 17:00:00


In [36]:
# Use date strings instead of milliseconds
url = "https://api.massive.com/v2/aggs/ticker/NVDA/range/5/minute/2025-11-20/2025-11-20"

response = requests.get(url, headers=headers, params=params, timeout=30)
data = response.json()

print(f"Using date strings:")
print(f"Status: {data.get('status')}")
print(f"Results: {data.get('resultsCount')}")

if data.get('results'):
    first = data['results'][0]
    last = data['results'][-1]
    print(f"First: {datetime.fromtimestamp(first['t']/1000)}")
    print(f"Last: {datetime.fromtimestamp(last['t']/1000)}")
    print(f"Total bars: {len(data['results'])}")

Using date strings:
Status: OK
Results: 192
First: 2025-11-20 17:00:00
Last: 2025-11-21 08:55:00
Total bars: 192


In [38]:
# Try from pre-market through close
start_ms = int(datetime(2025, 11, 20, 9, 0, 0).timestamp() * 1000)  # 4 AM EST
end_ms = int(datetime(2025, 11, 20, 21, 0, 0).timestamp() * 1000)   # 4 PM EST

url = f"https://api.massive.com/v2/aggs/ticker/NVDA/range/5/minute/{start_ms}/{end_ms}"

response = requests.get(url, headers=headers, params=params, timeout=30)
data = response.json()

print(f"Full day (pre-market to close):")
print(f"Status: {data.get('status')}")
print(f"Results: {data.get('resultsCount')}")
print(f"Query count: {data.get('queryCount')}")

if data.get('results'):
    # Show first 5 bars
    print(f"\nFirst 5 bars:")
    for i in range(min(5, len(data['results']))):
        bar = data['results'][i]
        dt = datetime.fromtimestamp(bar['t'] / 1000)
        # Convert to EST for display
        dt_est = dt - timedelta(hours=5)
        print(f"{i+1}. {dt} UTC = {dt_est.strftime('%I:%M %p')} EST - ${bar['c']:.2f}")

Full day (pre-market to close):
Status: OK
Results: 49
Query count: 245

First 5 bars:
1. 2025-11-20 17:00:00 UTC = 12:00 PM EST - $196.73
2. 2025-11-20 17:05:00 UTC = 12:05 PM EST - $196.68
3. 2025-11-20 17:10:00 UTC = 12:10 PM EST - $196.80
4. 2025-11-20 17:15:00 UTC = 12:15 PM EST - $196.85
5. 2025-11-20 17:20:00 UTC = 12:20 PM EST - $196.95


In [47]:
# Check the daily bar
start_ms = int(datetime(2025, 11, 18, 0, 0, 0).timestamp() * 1000)
end_ms = int(datetime(2025, 11, 19, 0, 0, 0).timestamp() * 1000)

url = f"https://api.massive.com/v2/aggs/ticker/NVDA/range/1/day/{start_ms}/{end_ms}"

response = requests.get(url, headers=headers, params=params, timeout=30)
data = response.json()

if data.get('results'):
    bar = data['results'][0]
    print(f"Daily bar:")
    print(f"Open:  ${bar['o']:.2f}")
    print(f"High:  ${bar['h']:.2f}")
    print(f"Low:   ${bar['l']:.2f}")
    print(f"Close: ${bar['c']:.2f}")
    print(f"Volume: {bar['v']:,.0f}")
    
    # Compare with your earlier official data
    print(f"\nExpected (official):")
    print(f"Open:  $195.95")
    print(f"High:  $196.00")
    print(f"Low:   $179.85")
    print(f"Close: $180.94")

Daily bar:
Open:  $185.97
High:  $189.00
Low:   $184.32
Close: $186.60
Volume: 173,623,379

Expected (official):
Open:  $195.95
High:  $196.00
Low:   $179.85
Close: $180.94


In [48]:
from datetime import datetime, timezone

# What Python thinks
dt_naive = datetime(2025, 11, 18, 0, 0, 0)
print(f"Naive datetime: {dt_naive}")
print(f"Timestamp: {dt_naive.timestamp()}")
print(f"As UTC: {datetime.fromtimestamp(dt_naive.timestamp())}")

# What it should be (explicit UTC)
dt_utc = datetime(2025, 11, 18, 0, 0, 0, tzinfo=timezone.utc)
print(f"\nUTC datetime: {dt_utc}")
print(f"Timestamp: {dt_utc.timestamp()}")
print(f"Difference: {(dt_utc.timestamp() - dt_naive.timestamp()) / 3600} hours")

Naive datetime: 2025-11-18 00:00:00
Timestamp: 1763395200.0
As UTC: 2025-11-18 00:00:00

UTC datetime: 2025-11-18 00:00:00+00:00
Timestamp: 1763424000.0
Difference: 8.0 hours


In [49]:
from datetime import datetime

# Nov 20, 2025 full trading day
start_time = datetime(2025, 11, 20, 14, 30, 0)  # 9:30 AM EST
end_time = datetime(2025, 11, 20, 21, 0, 0)     # 4:00 PM EST

bars = fetch_ohlc_bars("NVDA", start_time, end_time, "5min")

print(f"✅ Fetched {len(bars)} bars")
if bars:
    print(f"First bar: {bars[0]['timestamp']}")
    print(f"Last bar:  {bars[-1]['timestamp']}")
    
    # Should now get bars starting from 14:30 UTC (or at least closer!)
    expected_start = datetime(2025, 11, 20, 14, 30, 0)
    actual_start = bars[0]['timestamp']
    
    if actual_start <= datetime(2025, 11, 20, 15, 0, 0):
        print("\n✅ Timezone fix worked!")
    else:
        print(f"\n⚠️ Still offset: {actual_start} vs {expected_start}")

✅ Fetched 49 bars
First bar: 2025-11-20 17:00:00
Last bar:  2025-11-20 21:00:00

⚠️ Still offset: 2025-11-20 17:00:00 vs 2025-11-20 14:30:00
